# hola

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import f
from scipy.stats import chi2_contingency
from scipy.stats import normaltest
import statsmodels.api as sm
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'statsmodels'

In [69]:
def diagrama_barra(df_new):
    object_columns = df_new.select_dtypes(include="object").columns

    for column in object_columns:
        plt.figure(figsize=(10, 6))
        sns.barplot(
            x=df_new[column].value_counts().index, y=df_new[column].value_counts().values
        )
        plt.title(f"Diagrama de barras de {column}")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.xticks(rotation=45)
        for i, v in enumerate(df_new[column].value_counts().values):
            plt.text(i, v, str(v), ha="center", va="bottom")
        plt.show()

In [2]:
# Obtener la variable de entorno
ruta = "/home/mai_lavender/Escritorio/estadisticas/estadisticaG2/datos_proyecto.xlsx"

In [73]:
df = pd.read_excel(ruta)
columns_new = [
    "estudiante",
    "carrera",
    "sexo",
    "horario_tomado",
    "nota_estadistica",
    "nota_calculo",
    "nota_algebra",
    "nota_fund_prog",
]
df_new = df.copy()

df_new.columns = columns_new
df_new = df_new.drop(columns=["estudiante"])
# llevar la columna nota_estadistica a float
df_new["nota_estadistica"] = (
    df_new["nota_estadistica"].str.replace(",", ".").astype(float)
)
df_no_calculus = df_new[df_new["nota_calculo"].isna()]


In [74]:
# quiero eliminar la columna nota_calculo

df_no_calculus = df_no_calculus.drop(columns=["nota_calculo"])
df_no_calculus

In [75]:
# eliminar las filas que tienen la columna nota_calculo vacia
df_new = df_new.dropna(subset=["nota_calculo"])

# quiero sacar estudiante de mi dataframe


df_new.info()

In [76]:
df_new.std()

In [77]:
df_no_calculus.std()

In [78]:
df_new.describe()

In [79]:
df_no_calculus.describe()

In [80]:
# Obtener las columnas de tipo object

diagrama_barra(df_new)

In [81]:
diagrama_barra(df_no_calculus)

In [29]:
moda = df_new.mode()
moda

In [82]:
moda = df_no_calculus.mode()
moda

In [30]:
def boxplot(value):
    data = df_new.columns
    column_data = df_new[data[value]]
    if column_data.dtype in [np.int64, np.float64]:
        sns.set_style("whitegrid")
        fig, ax = plt.subplots(1, 1, figsize=(22, 2))
        sns.boxplot(x=column_data, color="#6E75CA", ax=ax)

        # Calculate and draw the mean
        mean = column_data.mean()
        ax.axvline(mean, color="#30db78", linestyle="--")
        # ax.text(mean, 0.75, 'Mean: {:.2f}'.format(mean), va='center', ha='center', backgroundcolor='w')

        # Calculate and draw the median
        median = column_data.median()
        ax.axvline(median, color="#c4db30", linestyle="-")
        # ax.text(median, 0.85, 'Median: {:.2f}'.format(median), va='center', ha='center', backgroundcolor='w')

        plt.title("Distribución de valores la variable " + data[value])
        plt.xlabel(data[value])
        plt.show()
        fig.tight_layout()


def histogram_f_fn_x(df, column: str, b: int):

    # Configurar el estilo del gráfico
    sns.set_style("whitegrid")
    # Crear el histograma y obtener los valores de las barras
    counts, bins, patches = plt.hist(df[column], bins=b, color="#6E75CA")
    # Añadir título y etiquetas a los ejes
    plt.title(f"Histograma de Frecuencia Absoluta de {column}")
    plt.xlabel(column)
    plt.ylabel("Frecuencia")
    # Modificar el eje x para mostrar el intervalo de cada barra
    plt.xticks(bins, [f"{bin:.2f}-{bins[i+1]:.2f}" for i, bin in enumerate(bins[:-1])])
    # Para cada barra: colocar una etiqueta con el valor de la altura de la barra
    for count, bin, patch in zip(counts, bins, patches):
        plt.text(bin, patch.get_height(), f"{count:.0f}", color="black", ha="left")
    # Mostrar el gráfico
    plt.show()

    # Configurar el estilo del gráfico
    sns.set_style("whitegrid")
    # Crear el histograma de frecuencia relativa y obtener los valores de las barras
    counts, bins, patches = plt.hist(df[column], bins=b, color="#6E75CA", density=True)
    # Añadir título y etiquetas a los ejes
    plt.title(f"Histograma de Frecuencia Relativa de {column}")
    plt.xlabel(column)
    plt.ylabel("Frecuencia Relativa")
    # Modificar el eje x para mostrar el intervalo de cada barra
    plt.xticks(bins, [f"{bin:.2f}-{bins[i+1]:.2f}" for i, bin in enumerate(bins[:-1])])
    # Para cada barra: colocar una etiqueta con el valor de la altura de la barra
    for count, bin, patch in zip(counts, bins, patches):
        plt.text(bin, patch.get_height(), f"{count:.3f}", color="black", ha="left")
    # Mostrar el gráfico
    plt.show()


def boxplot_a(value):
    data = df_new.columns
    column_data = df_new[data[value]]
    if column_data.dtype in [np.int64, np.float64]:
        sns.set_style("whitegrid")
        fig, ax = plt.subplots(1, 1, figsize=(22, 2))
        sns.boxplot(x=column_data, color="#6E75CA")

        # Calculate quartiles and limits
        q1 = column_data.quantile(0.25)
        q2 = column_data.quantile(0.5)
        q3 = column_data.quantile(0.75)
        iqr = q3 - q1
        lower_limit = q1 - 1.5 * iqr
        upper_limit = q3 + 1.5 * iqr

        # Calculate and draw the mean
        mean = column_data.mean()
        ax.axvline(mean, color="#30db78", linestyle="--")
        # ax.text(mean, 0.75, 'Mean: {:.2f}'.format(mean), va='center', ha='center', backgroundcolor='w')

        # Calculate and draw the median
        median = column_data.median()
        ax.axvline(median, color="#c4db30", linestyle="-")
        # ax.text(median, 0.85, 'Median: {:.2f}'.format(median), va='center', ha='center', backgroundcolor='w')

        # Add quartile values and limits to the plot
        ax.text(0.06, 0.90, f"Lower Limit: {lower_limit:.2f}", transform=ax.transAxes)
        ax.text(0.06, 0.80, f"Upper Limit: {upper_limit:.2f}", transform=ax.transAxes)

        plt.title("Distribución de valores la variable " + data[value])
        plt.xlabel(data[value])
        plt.show()
        fig.tight_layout()


def histogram_f_fn(df, column: str, b: int):

    # Configurar el estilo del gráfico
    sns.set_style("whitegrid")
    # Crear el histograma y obtener los valores de las barras
    counts, bins, patches = plt.hist(df[column], bins=b, color="#6E75CA")
    # Añadir título y etiquetas a los ejes
    plt.title(f"Histograma de Frecuencia Absoluta de {column}")
    plt.xlabel(column)
    plt.ylabel("Frecuencia")
    # Para cada barra: colocar una etiqueta con el valor de la altura de la barra
    for count, bin, patch in zip(counts, bins, patches):
        plt.text(bin, patch.get_height(), f"{count:.0f}", color="black", ha="left")
    # Mostrar el gráfico
    plt.show()

    # Calcular la frecuencia relativa
    total_data = len(df)
    relative_freq = counts / total_data

    # Configurar el estilo del gráfico
    sns.set_style("whitegrid")
    # Crear el histograma de frecuencia relativa y obtener los valores de las barras
    _, _, patches = plt.hist(df[column], bins=b, color="#6E75CA", density=True)
    # Añadir título y etiquetas a los ejes
    plt.title(f"Histograma de Frecuencia Relativa de {column}")
    plt.xlabel(column)
    plt.ylabel("Frecuencia Relativa")
    # Para cada barra: colocar una etiqueta con el valor de la altura de la barra
    for relative_freq, bin, patch in zip(relative_freq, bins, patches):
        plt.text(
            bin, patch.get_height(), f"{relative_freq:.3f}", color="black", ha="left"
        )
    # Mostrar el gráfico
    plt.show()


def tabla_frec(df, column: str):
    max_val = df[column].max()
    min_val = df[column].min()
    rango = max_val - min_val
    k = int(1 + 3.322 * np.log10(df[column].count()))
    print(k)
    AK = round(rango / k, 2)
    n = df[column].count()
    tb = pd.DataFrame(columns=["ordinal", "k", "mk", "f", "f/n", "F", "F/n"])
    tb["ordinal"] = range(1, k + 1)
    tb["k"] = [
        f"[{round(min_val + AK * i, 2)} - {round(min_val + AK * (i+1), 2)}]"
        for i in range(k)
    ]
    tb["mk"] = [(min_val + AK * i + min_val + AK * (i + 1)) / 2 for i in range(k)]
    tb["f"] = [
        (
            (df[column] >= min_val + AK * i)
            & (
                df[column] < min_val + AK * (i + 1)
                if i < k - 1
                else df[column] <= min_val + AK * (i + 1)
            )
        ).sum()
        for i in range(k)
    ]
    tb["f/n"] = round(tb["f"] / n, 3)
    tb["F"] = tb["f"].cumsum()
    tb["F/n"] = round(tb["F"] / n, 3)
    return tb


def scatter_plots_with_linear_function(df):
    columns = df.columns

    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            x = columns[i]
            y = columns[j]

            plt.figure(figsize=(8, 6))
            plt.scatter(df[x], df[y], color="#22BABB")
            plt.title(f"Scatter Plot: {x} vs {y}")
            plt.xlabel(x)
            plt.ylabel(y)

            # Calculate correlation coefficient
            corr_coef = np.corrcoef(df[x], df[y])[0, 1]

            # Add correlation coefficient as text annotation
            plt.text(
                df[x].min(),
                df[y].max(),
                f"Corr: {corr_coef:.2f}",
                color="black",
                ha="left",
            )

            # Add linear function
            m, b = np.polyfit(df[x], df[y], 1)
            plt.plot(df[x], m * df[x] + b, color="#EF8026")

            plt.show()


def scatter_plots(df):
    columns = df.columns

    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            x = columns[i]
            y = columns[j]

            plt.figure(figsize=(8, 6))
            plt.scatter(df[x], df[y], color="#6E75CA")
            plt.title(f"Scatter Plot: {x} vs {y}")
            plt.xlabel(x)
            plt.ylabel(y)

            # Calculate correlation coefficient
            corr_coef = np.corrcoef(df[x], df[y])[0, 1]

            # Add correlation coefficient as text annotation
            plt.text(
                df[x].min(),
                df[y].max(),
                f"Corr: {corr_coef:.2f}",
                color="black",
                ha="left",
            )

            plt.show()


def boxplot_x2(df, column: str, column2: str):
    sns.boxplot(
        data=df,
        x=column,
        y=column2,
        order=["male", "female"],
        palette=["#6E75CA", "#F2CB05", "#6DAFC9", "#EF8026", "#CC99FF"],
    )
    plt.title(f"Diagrama de cajas de {column2} por {column}")
    plt.xlabel(column)
    plt.ylabel(column2)
    plt.show()


def boxplot_x3(df, column: str, column2: str):
    sns.boxplot(
        data=df,
        x=column,
        y=column2,
        order=["no", "yes"],
        palette=["#6E75CA", "#F2CB05", "#6DAFC9", "#EF8026", "#CC99FF"],
    )
    plt.title(f"Diagrama de cajas de {column2} por {column}")
    plt.xlabel(column)
    plt.ylabel(column2)
    plt.show()

In [83]:
dataTypes = df_no_calculus.dtypes
for i in range(dataTypes.size):
    boxplot_a(i)

In [84]:
dataTypes = df_new.dtypes
for i in range(dataTypes.size):
    boxplot_a(i)

In [85]:
estadistica = tabla_frec(df_new, "nota_estadistica")
estadistica

In [86]:
calculo = tabla_frec(df_new, "nota_calculo")
calculo

In [87]:
algebra = tabla_frec(df_new, "nota_algebra")
algebra

In [88]:
programacion = tabla_frec(df_new, "nota_fund_prog")
programacion

In [89]:
histogram_f_fn(df_new, "nota_estadistica", 10)
histogram_f_fn(df_new, "nota_calculo", 10)
histogram_f_fn(df_new, "nota_algebra", 10)
histogram_f_fn(df_new, "nota_fund_prog", 10)

In [90]:
df_new_numerics = df_new.select_dtypes(include=[np.number])
df_new_numerics.corr()

In [91]:
plt.figure(figsize=(5, 5))
sns.heatmap(
    df_new_numerics.corr(),
    # cmap = 'coolwarm',
    annot=True,
    square=True,
)

In [92]:
MT = df_new.select_dtypes(exclude=["object", "bool"]).copy()

sns.pairplot(MT, plot_kws={"color": "#6E75CA"}, diag_kws={"color": "#6E75CA"})

In [93]:
scatter_plots_with_linear_function(MT)

In [ ]:
boxplot_x2(df_new, "Sexo", "Edad")
boxplot_x2(df_new, "Sexo", "BMI")
boxplot_x2(df_new, "Sexo", "Costos")

In [ ]:
boxplot_x3(df_new, "Hábito de fumar", "Edad")
boxplot_x3(df_new, "Hábito de fumar", "BMI")
boxplot_x3(df_new, "Hábito de fumar", "Costos")

In [ ]:
def generate_contingency_tables(df):
    categorical_vars = [
        var
        for var in df.columns
        if df[var].dtype == "object" or df[var].dtype == "bool"
    ]

    contingency_tables = {}

    for i in range(len(categorical_vars)):
        for j in range(i + 1, len(categorical_vars)):
            var1 = categorical_vars[i]
            var2 = categorical_vars[j]

            if (
                f"{var1} vs {var2}" not in contingency_tables
                and f"{var2} vs {var1}" not in contingency_tables
            ):
                contingency_table = pd.crosstab(
                    index=df[var1], columns=df[var2], margins=True, margins_name="Total"
                )
                contingency_tables[f"{var1} vs {var2}"] = contingency_table
                # Guardar la tabla de contingencia en un DataFrame separado
                globals()[f"{var1}_vs_{var2}_table"] = contingency_table

    return contingency_tables


generate_contingency_tables(df_new)

## Prueba hipotesis: diferencia de medias de los costos médicos con respecto a habitos de fumar

H0 = La media de los costos médicos es igual entre fumadores y mo fumadores

H1 = La media de los costos médicos es diferente entre fumadores y mo fumadores

Supuestos: X1,X2 población normal e independientes 𝜎12,𝜎22 desconocido n1,n2 grande

In [ ]:
no_fumador = df_new[df_new["Hábito de fumar"] == "no"]["Costos"].tolist()
fumador = df_new[df_new["Hábito de fumar"] == "yes"]["Costos"].tolist()
print(len(no_fumador))
print(len(fumador))

In [49]:
# Nivel de significancia
nivel_significancia = 0.05

# Realizamos la prueba t de Student para muestras independientes
t_stat, p_valor = stats.ttest_ind(no_fumador, fumador, equal_var=False)

# Imprimimos los resultados
print("Estadístico t:", t_stat)
print("Valor p:", p_valor)

# Interpretación del resultado
if p_valor < nivel_significancia:
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

In [ ]:
no_saludable = df_new[(df_new["BMI"] < 18.5) | (df_new["BMI"] > 24.9)]["BMI"]
saludable = df_new[(df_new["BMI"] >= 18.5) & (df_new["BMI"] <= 24.9)]["BMI"]
print(df_new["BMI"].count())

In [ ]:
# Contar los individuos en cada categoría
n_no_saludable = no_saludable.count()
n_saludable = saludable.count()
n_total = df_new["BMI"].count()

# Proporción observada de no saludables
prop_no_saludable = n_no_saludable / n_total

# Proporción saludable (hipótesis nula: 50%)
prop_hipotesis_nula = 0.5

# Realizar la prueba de proporciones z
z_stat, p_valor = proportions_ztest(
    count=n_no_saludable, nobs=n_total, value=prop_hipotesis_nula, alternative="larger"
)

# Imprimir resultados
print("Estadístico Z:", z_stat)
print("Valor p:", p_valor)

# Interpretar resultados
nivel_significancia = 0.05
if p_valor < nivel_significancia:
    print(
        "Rechazamos la hipótesis nula: la proporción de BMI no saludable es mayor que la saludable."
    )
else:
    print(
        "No se rechaza la hipótesis nula: no hay evidencia suficiente para decir que la proporción no saludable es mayor."
    )

In [ ]:
hombres = df_new[df_new["Sexo"] == "male"]["Costos"].tolist()
mujeres = df_new[df_new["Sexo"] == "female"]["Costos"].tolist()
print(len(hombres))
print(len(mujeres))

In [56]:
# Nivel de significancia
nivel_significancia = 0.05

# Realizamos la prueba t de Student para muestras independientes
t_stat, p_valor = stats.ttest_ind(hombres, mujeres, equal_var=False)

# Imprimimos los resultados
print("Estadístico t:", t_stat)
print("Valor p:", p_valor)

# Interpretación del resultado
if p_valor < nivel_significancia:
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")